In [1]:
import os

from sqlalchemy.orm import sessionmaker

from cctmodels import create_table, sdb_connect,  mdb_connect

from cctmodels import Patient

from cctmodels import TempItemInfo, TempItemExecInfo

from cctmodels import CheckReport, CheckReportInfo, CheckReportItem

from cctmodels import FeeRecord, FeeItem

from cctmodels import EPR as EPRecord
from cctmodels import OpEMRData, RANinfo, FirstCorinfo, DailyRecdata, GetDisRecordinfo, DeathRecordinfo, OperRecdata, hourRec

engine = mdb_connect()

create_table(engine)
Session = sessionmaker(bind = engine)
session = Session()

In [2]:
## The number of Patient
import pandas as pd
import xmltodict

df = pd.read_json('data/xml_table.json', orient = 'split')
df

,Id,RecipeID,PatientID,JobNumber,RegiNumber,JobOrgan_Id,DataType,Data
0,6702,281,31571,3168507,130256,1,5,<EPRdata><RANinfo><A1>入院记录</A1><A2>入院记录</A2><B...
1,6716,475,31571,1653681,130256,1,5,<EPRdata><RANinfo><A1>入院记录</A1><A2>入院记录</A2><B...
2,11584,2088,26309,3789068,945512,1,7,<Request><HisID/><Name/><Sex>女</Sex><Age>49岁</...
3,63605,1182,50866,4228718,52032,1,7,<Request><HisID/><Name/><Sex>女</Sex><Age>29岁</...
4,6707,395,31571,2115881,130256,1,3,<Request><HisID/><Name/><Sex>男</Sex><Age>82岁</...
5,63781,1231,50911,4212726,126590,1,7,<Request><HisID/><Name/><Sex>男</Sex><Age>65岁</...
6,63897,1263,52818,4212247,996188,1,7,<Request><HisID/><Name/><Sex>男</Sex><Age>73岁</...
7,6718,629,31571,764829,130256,1,3,<Request><HisID/><Name/><Sex>男</Sex><Age>82岁</...
8,6728,672,24531,4310335,1064503,1,2,<Request><Name/><Sex>男</Sex><Age>40岁</Age><InA...
9,64023,1297,48554,4203737,420006,1,7,<Request><HisID/><Name/><Sex>女</Sex><Age>48岁</...


In [3]:
def session_add(session, item):
    try:
        session.add(item)
    except:
        session.rollback()
        raise

## Type 2 FeeRecords

In [4]:
dt = df[df['DataType'] == 2].iloc[1]
dt_dict = dt.to_dict()
dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
dt_dict

{'Data': OrderedDict([('Request',
               OrderedDict([('Name', None),
                            ('Sex', '男'),
                            ('Age', '11岁'),
                            ('InAdmDate', '2017-07-23'),
                            ('CurLoc', '耳鼻喉科门诊'),
                            ('TotalAmount', '123.05'),
                            ('YBAmount', '86.67'),
                            ('YJAmount', '0'),
                            ('FeeList',
                             OrderedDict([('FeeList',
                                           [OrderedDict([('PAADM', None),
                                                         ('EMCCate', '诊查费'),
                                                         ('TARIItem',
                                                          '主任医师门诊诊查费'),
                                                         ('UnitPrice', '8'),
                                                         ('BillQty', '1'),
                                     

In [5]:
int(dt_dict['PatientID'])

25562

In [6]:
def session_add(session, item):
    try:
        session.add(item)
    except:
        session.rollback()
        raise

In [7]:
patt = session.query(Patient).filter_by(PatientID = dt_dict['PatientID']).first()

if patt == None:
    patt = Patient(PatientID = dt_dict['PatientID'])
    session_add(session, patt)
patt 

AttributeError: 'numpy.int64' object has no attribute 'translate'

In [ ]:
Data = dt_dict['Data']
Data

In [ ]:
## Only one

feeRec = Data['Request']
feeItems = feeRec.pop('FeeList')

In [10]:
feeRec

OrderedDict([('Name', None),
             ('Sex', '男'),
             ('Age', '11岁'),
             ('InAdmDate', '2017-07-23'),
             ('CurLoc', '耳鼻喉科门诊'),
             ('TotalAmount', '123.05'),
             ('YBAmount', '86.67'),
             ('YJAmount', '0')])

In [11]:
feeItems['FeeList']

[OrderedDict([('PAADM', None),
              ('EMCCate', '诊查费'),
              ('TARIItem', '主任医师门诊诊查费'),
              ('UnitPrice', '8'),
              ('BillQty', '1'),
              ('Amount', '8'),
              ('BillType', '2'),
              ('BillStatus', '已付费')]),
 OrderedDict([('PAADM', None),
              ('EMCCate', '西药费'),
              ('TARIItem', '0.9%氯化钠注射液(软袋)l×1袋]'),
              ('UnitPrice', '1.95'),
              ('BillQty', '1'),
              ('Amount', '1.95'),
              ('BillType', '2'),
              ('BillStatus', '已付费')]),
 OrderedDict([('PAADM', None),
              ('EMCCate', '治疗费'),
              ('TARIItem', '耵聍冲洗'),
              ('UnitPrice', '12'),
              ('BillQty', '1'),
              ('Amount', '12'),
              ('BillType', '2'),
              ('BillStatus', '已付费')]),
 OrderedDict([('PAADM', None),
              ('EMCCate', '治疗费'),
              ('TARIItem', '耳正负压治疗'),
              ('UnitPrice', '12'),
              ('BillQty'

In [12]:
dt = df[df['DataType'] == 2].iloc[1]
dt_dict = dt.to_dict()
dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
dt_dict['PatientID']

25562

In [13]:
patt = session.query(Patient).filter_by(PatientID = int(dt_dict['PatientID'])).first()
if patt == None:
    patt = Patient(PatientID = int(dt_dict['PatientID']))

patt.PatientID

25562

In [14]:
if dt_dict['DataType'] == 2 or dt_dict['DataType'] == 6:
    feeRec = dt_dict['Data']['Request']
    feeItems = feeRec.pop('FeeList')
    feeRec = FeeRecord(**feeRec)
    
    for fitem in feeItems['FeeList']:
        feeItm = FeeItem(**fitem)
        feeRec.FeeItems.append(feeItm)
        session_add(session, feeItm )
        
    session_add(session, feeRec)
    patt.FeeRecords.append(feeRec)
    session_add(session, patt)
    
session.commit()


In [ ]:
def WriteFeeList(dt_dict, session, patt):
    feeRec = dt_dict['Data']['Request']
    if 'FeeList' in feeRec:
        feeItems = feeRec.pop('FeeList')
        feeItems = feeItems['FeeList'] # feeItems may be 1 item, or a list of items
    else:
        feeItems = []

    feeRec = FeeRecord(**feeRec)
    if type(feeItems) != list:
        feeItems = [feeItems]
    for fitem in feeItems:
        feeItm = FeeItem(**fitem)
        feeRec.FeeItems.append(feeItm)
        session_add(session, feeItm )
    session_add(session, feeRec)
    patt.FeeRecords.append(feeRec)
    session_add(session, patt)
    

dt = df[df['DataType'] == 2].iloc[11]
dt_dict = dt.to_dict()
dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
dt_dict

In [ ]:
patt = session.query(Patient).filter_by(PatientID = int(dt_dict['PatientID'])).first()
if patt == None:
    patt = Patient(PatientID = int(dt_dict['PatientID']))

if dt_dict['DataType'] == 2 or dt_dict['DataType'] == 6:
    WriteFeeList(dt_dict, session, patt)
    
session.commit()

In [ ]:
dataType = df[df['DataType'] == 6]
for ind, dt in dataType.iterrows():
    dt_dict = dt.to_dict()
    dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
    
    patt = session.query(Patient).filter_by(PatientID = int(dt_dict['PatientID'])).first()
    if patt == None:
        patt = Patient(PatientID = int(dt_dict['PatientID']))

    if dt_dict['DataType'] == 2 or dt_dict['DataType'] == 6:
        WriteFeeList(dt_dict, session, patt)
    
session.commit()

## Type 3 CheckReport

In [ ]:
dt = df[df['DataType'] == 3].iloc[1]
dt_dict = dt.to_dict()
dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
dt_dict

In [ ]:
patt = session.query(Patient).filter_by(PatientID = int(dt_dict['PatientID'])).first()
if patt == None:
    patt = Patient(PatientID = int(dt_dict['PatientID']))
    
#from cctmodels import CheckReport, CheckReportInfo, CheckReportItem

In [6]:
patt.PatientID

31571

In [7]:
RPT = dt_dict['Data']['Request']
RPTInfos = RPT.pop('ReportInfo')
RPT

OrderedDict([('HisID', None),
             ('Name', None),
             ('Sex', '男'),
             ('Age', '82岁'),
             ('InAdmDate', '2014-06-10'),
             ('CurLoc', '心血管内科'),
             ('CurBed', 'JC04'),
             ('CurWard', 'NYBQ-内一病区'),
             ('Diagnosis', '高血压病3级极高危，腹泻查因')])

In [13]:
CckRpt = CheckReport(**RPT)
CckRpt.CurLoc

'心血管内科'

In [14]:
#print(type(RPTInfos))
type(RPTInfos['ReportInfo'])

list

In [25]:
RPTInfos = RPTInfos['ReportInfo']
RPTInfos

[OrderedDict([('ArcName', '肝功八项'),
              ('Date', '2014-06-10'),
              ('Loc', '医学检验科'),
              ('ReportDate', '2014-06-11 00:09:56'),
              ('ReportDoctor', '蔡钦泉'),
              ('Type', '检验'),
              ('OrderID', '764926||11'),
              ('StudyNo', '51000683586'),
              ('ReportItem',
               OrderedDict([('ReportItem',
                             [OrderedDict([('ItemName', '总蛋白'),
                                           ('ExamDetail', '66.1'),
                                           ('RefRanger', '65--85g/L'),
                                           ('Unit', 'g/L')]),
                              OrderedDict([('ItemName', '总胆红素'),
                                           ('ExamDetail', '13.6'),
                                           ('RefRanger', '3.4--17.1umol/L'),
                                           ('Unit', 'umol/L')]),
                              OrderedDict([('ItemName', '谷丙转氨酶'),
              

In [26]:
RPTInfo = RPTInfos[4]
RPTItems = RPTInfo.pop('ReportItem')

In [27]:
RPTInfo

OrderedDict([('ArcName', '糖化血红蛋白'),
             ('Date', '2014-06-10'),
             ('Loc', '医学检验科'),
             ('ReportDate', '2014-06-11 00:14:48'),
             ('ReportDoctor', '蔡钦泉'),
             ('Type', '检验'),
             ('OrderID', '764926||15'),
             ('StudyNo', '51000683587')])

In [18]:
CckRptInfo = CheckReportInfo(**RPTInfo)
CckRptInfo.Loc

'医学检验科'

In [28]:
RPTItems = RPTItems['ReportItem']
RPTItems

OrderedDict([('ItemName', '糖化血红蛋白'),
             ('ExamDetail', '5.6'),
             ('RefRanger', '4-6％'),
             ('Unit', '％')])

In [20]:
RPTItem = RPTItems[0]
RPTItem

OrderedDict([('ItemName', '总蛋白'),
             ('ExamDetail', '66.1'),
             ('RefRanger', '65--85g/L'),
             ('Unit', 'g/L')])

In [21]:
CckRptItem = CheckReportItem(**RPTItem)
CckRptItem.ItemName

'总蛋白'

In [22]:
CckRptInfo.CheckReportItems.append(CckRptItem)
CckRpt.CheckReportInfos.append(CckRptInfo)
patt.CheckReports.append(CckRpt)

In [45]:
def WriteCheckReport(dt_dict, session, patt):
    RPT = dt_dict['Data']['Request']
    if 'ReportInfo' in RPT:
        RPTInfos = RPT.pop('ReportInfo')
        RPTInfos = RPTInfos['ReportInfo']
    else:
        print(RPT)
        RPTInfos = []
        
    CckRpt = CheckReport(**RPT)
    
    if type(RPTInfos) != list:
        print('Only One')
        RPTInfos = [ RPTInfos ] 

    for RPTInfo in RPTInfos:
        if 'ReportItem' in RPTInfo:
            RPTItems = RPTInfo.pop('ReportItem')
            RPTItems = RPTItems['ReportItem']
        else:
            print(RPTInfo)
            RPTItems = []
            
        CckRptInfo = CheckReportInfo(**RPTInfo)
        
        if type(RPTItems) != list:
            print('Only One')
            RPTItems = [ RPTItems]
        for RPTItem in RPTItems:
            CckRptItem = CheckReportItem(**RPTItem)
            CckRptInfo.CheckReportItems.append(CckRptItem)
            session_add(session, CckRptItem)
            
        CckRpt.CheckReportInfos.append(CckRptInfo)
        session_add(session, CckRptInfo)
        
    patt.CheckReports.append(CckRpt)
    session_add(session, CckRpt)

In [56]:
dt = df[df['DataType'] == 3].iloc[1]
dt_dict = dt.to_dict()
dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
dt_dict

patt = session.query(Patient).filter_by(PatientID = int(dt_dict['PatientID'])).first()
if patt == None:
    patt = Patient(PatientID = int(dt_dict['PatientID']))
    
    
WriteCheckReport(dt_dict, session, patt)


session_add(session, patt)
session.commit()


Only One
Only One


In [57]:
session.commit()

In [58]:
dataType = df[df['DataType'] == 3 ]
for ind, dt in dataType.iterrows():
    dt_dict = dt.to_dict()
    dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
    
    patt = session.query(Patient).filter_by(PatientID = int(dt_dict['PatientID'])).first()
    if patt == None:
        patt = Patient(PatientID = int(dt_dict['PatientID']))

    if dt_dict['DataType'] in [3, 7 ] :
        WriteCheckReport(dt_dict, session, patt)
        
    session_add(session, patt)
    session.commit()
    
session.commit()

Only One
Only One
Only One
Only One
Only One
Only One
OrderedDict([('HisID', None), ('Name', None), ('Sex', '男'), ('Age', '11岁'), ('InAdmDate', '2017-07-23'), ('CurLoc', '耳鼻喉科门诊'), ('CurBed', None), ('CurWard', None), ('Diagnosis', '慢性扁桃体炎')])
OrderedDict([('HisID', None), ('Name', None), ('Sex', '男'), ('Age', '41岁'), ('InAdmDate', '2017-07-23'), ('CurLoc', '消化内科门诊'), ('CurBed', None), ('CurWard', None), ('Diagnosis', '反流性食管炎')])
Only One
Only One
OrderedDict([('HisID', None), ('Name', None), ('Sex', '女'), ('Age', '30岁'), ('InAdmDate', '2017-07-21'), ('CurLoc', '妇科门诊'), ('CurBed', None), ('CurWard', None), ('Diagnosis', '子宫颈炎')])
OrderedDict([('HisID', None), ('Name', None), ('Sex', '女'), ('Age', '52岁'), ('InAdmDate', '2017-07-21'), ('CurLoc', '妇科门诊'), ('CurBed', None), ('CurWard', None), ('Diagnosis', '绝经期和女性更年期状态')])
Only One
Only One
OrderedDict([('HisID', None), ('Name', None), ('Sex', '女'), ('Age', '48岁'), ('InAdmDate', '2017-07-21'), ('CurLoc', '急诊外科'), ('CurBed', None), ('CurWar

Only One
OrderedDict([('HisID', None), ('Name', None), ('Sex', '女'), ('Age', '25岁'), ('InAdmDate', '2017-07-15'), ('CurLoc', '急诊外科'), ('CurBed', None), ('CurWard', None), ('Diagnosis', None)])
OrderedDict([('HisID', None), ('Name', None), ('Sex', '男'), ('Age', '19岁'), ('InAdmDate', '2017-03-21'), ('CurLoc', '急诊外科'), ('CurBed', None), ('CurWard', None), ('Diagnosis', None)])
Only One
OrderedDict([('HisID', None), ('Name', None), ('Sex', '女'), ('Age', '27岁'), ('InAdmDate', '2017-08-04'), ('CurLoc', '体检保健科'), ('CurBed', None), ('CurWard', None), ('Diagnosis', None)])
Only One
Only One
OrderedDict([('HisID', None), ('Name', None), ('Sex', '男'), ('Age', '4岁'), ('InAdmDate', '2017-07-21'), ('CurLoc', '儿科门诊'), ('CurBed', None), ('CurWard', None), ('Diagnosis', '急性咽喉炎')])
OrderedDict([('HisID', None), ('Name', None), ('Sex', '女'), ('Age', '29岁'), ('InAdmDate', '2017-07-23'), ('CurLoc', '罗江萍名中医工作室'), ('CurBed', None), ('CurWard', None), ('Diagnosis', '肝胃不和 ,月经过少 ,带下病 ,维生素Ｅ缺乏')])
Only One
Only O

Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
O

Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
OrderedDict([('HisID', None), ('Name', None), ('Sex', '女'), ('Age', '47岁'), ('InAdmDate', '2017-06-19'), ('CurLoc', '伽玛刀科'), ('CurBed', '18'), ('CurWard', 'ZHKBQ-综合科病区'), ('Diagnosis', '斜坡脑脑瘤术后残留')])
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One

Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
Only One
OrderedDict([('HisID', None), ('Name', None), ('Sex', '女'), ('Age', '62岁'), ('InAdmDate', '2014-06-13'), ('CurLoc', '血液透析室住院'), ('CurBed', 'XT13'), ('CurWard', 'XY

## Type 4 TempItemInfo

In [4]:
dt = df[df['DataType'] == 4].iloc[1]
dt_dict = dt.to_dict()
dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
dt_dict

{'Data': OrderedDict([('Request',
               OrderedDict([('TempItemInfo',
                             OrderedDict([('TempItemInfo',
                                           [OrderedDict([('OEORDRowId',
                                                          '4309328||7'),
                                                         ('ItemName',
                                                          '布洛芬混悬液(恬倩)[2g:100ml×1瓶]'),
                                                         ('ItemCat', None),
                                                         ('InstrDR', '口服10ml'),
                                                         ('DoseQty', '口服10ml'),
                                                         ('PHFreqDR', '一次'),
                                                         ('Remark', '1'),
                                                         ('Status', '已执行'),
                                                         ('PrescNo', None),
                      

In [50]:
# from cctmodels import Patient, TempItemInfo, TempItemExecInfo

dt_dict['Data']['Request']

OrderedDict([('TempItemInfo',
              OrderedDict([('TempItemInfo',
                            [OrderedDict([('OEORDRowId', '4309328||7'),
                                          ('ItemName',
                                           '布洛芬混悬液(恬倩)[2g:100ml×1瓶]'),
                                          ('ItemCat', None),
                                          ('InstrDR', '口服10ml'),
                                          ('DoseQty', '口服10ml'),
                                          ('PHFreqDR', '一次'),
                                          ('Remark', '1'),
                                          ('Status', '已执行'),
                                          ('PrescNo', None),
                                          ('Priority', None),
                                          ('ToldType', '临时医嘱'),
                                          ('YJDate', '2017-08-04'),
                                          ('Date', None),
                                          

In [52]:
TMPItems = dt_dict['Data']['Request']['TempItemInfo']['TempItemInfo']
TMPItems

[OrderedDict([('OEORDRowId', '4309328||7'),
              ('ItemName', '布洛芬混悬液(恬倩)[2g:100ml×1瓶]'),
              ('ItemCat', None),
              ('InstrDR', '口服10ml'),
              ('DoseQty', '口服10ml'),
              ('PHFreqDR', '一次'),
              ('Remark', '1'),
              ('Status', '已执行'),
              ('PrescNo', None),
              ('Priority', None),
              ('ToldType', '临时医嘱'),
              ('YJDate', '2017-08-04'),
              ('Date', None),
              ('Loc', '发热门诊'),
              ('RecLoc', '门诊西药房'),
              ('Price', '5.8'),
              ('Unit', None),
              ('RecLocID', None),
              ('StDate', '2017-08-04'),
              ('EtDate', None),
              ('RecLocDesc', None),
              ('NeedExecTime', None),
              ('CommonName', '布洛芬'),
              ('YPFlag', '1')]),
 OrderedDict([('OEORDRowId', '4309328||6'),
              ('ItemName', '氨酚伪麻美芬片/氨麻美敏片Ⅱ(日夜百服咛)[0.5g×12片×1盒]'),
              ('ItemCat', None),
  

In [53]:
TMPItem = TMPItems[4]
TMPItem

OrderedDict([('OEORDRowId', '4309328||3'),
             ('ItemName', '(加急)★血常规'),
             ('ItemCat', None),
             ('InstrDR', None),
             ('DoseQty', None),
             ('PHFreqDR', None),
             ('Remark', '1'),
             ('Status', None),
             ('PrescNo', None),
             ('Priority', None),
             ('ToldType', '临时医嘱'),
             ('YJDate', '2017-08-04'),
             ('Date', None),
             ('Loc', '发热门诊'),
             ('RecLoc', '医学检验科'),
             ('Price', '20'),
             ('Unit', None),
             ('RecLocID', None),
             ('StDate', '2017-08-04'),
             ('EtDate', None),
             ('RecLocDesc', None),
             ('NeedExecTime', None),
             ('TempItemExecInfoS',
              OrderedDict([('TempItemExecInfo',
                            OrderedDict([('ExecTime', '2017-08-04 11:01:37'),
                                         ('NeedExecTime',
                                          '

In [4]:
def WriteTempItem(dt_dict, session, patt):
    TMPItems = dt_dict['Data']['Request']['TempItemInfo']['TempItemInfo']
    
    if type(TMPItems) != list:
        print('Only One' )
        TMPItems = [TMPItems]
        
        
    for TMPItem in TMPItems: 
        
        if 'TempItemExecInfoS' in TMPItem:
            TMPIExecs = TMPItem.pop('TempItemExecInfoS')
            TMPIExecs = TMPIExecs['TempItemExecInfo']
            if type(TMPIExecs) != list:
                TMPIExecs = [TMPIExecs]
        else:
            TMPIExecs = []

        TmpItmInfo = TempItemInfo(**TMPItem)

        for TMPIExec in TMPIExecs:
            TmpItmExec = TempItemExecInfo(**TMPIExec)

            TmpItmInfo.TempItemExecInfos.append(TmpItmExec)
            session_add(session, TmpItmExec)

        patt.TempItemInfos.append(TmpItmInfo)
        session_add(session, TmpItmInfo)
    

In [5]:
dt = df[df['DataType'] == 4].iloc[1]
dt_dict = dt.to_dict()
dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
dt_dict

{'Data': OrderedDict([('Request',
               OrderedDict([('TempItemInfo',
                             OrderedDict([('TempItemInfo',
                                           [OrderedDict([('OEORDRowId',
                                                          '4309328||7'),
                                                         ('ItemName',
                                                          '布洛芬混悬液(恬倩)[2g:100ml×1瓶]'),
                                                         ('ItemCat', None),
                                                         ('InstrDR', '口服10ml'),
                                                         ('DoseQty', '口服10ml'),
                                                         ('PHFreqDR', '一次'),
                                                         ('Remark', '1'),
                                                         ('Status', '已执行'),
                                                         ('PrescNo', None),
                      

In [6]:
patt = session.query(Patient).filter_by(PatientID = int(dt_dict['PatientID'])).first()
if patt == None:
    patt = Patient(PatientID = int(dt_dict['PatientID']))
    
    
WriteTempItem(dt_dict, session, patt)


session_add(session, patt)
session.commit()


In [7]:
dataType = df[df['DataType'] == 4 ]
for ind, dt in dataType.iterrows():
    dt_dict = dt.to_dict()
    dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
    
    patt = session.query(Patient).filter_by(PatientID = int(dt_dict['PatientID'])).first()
    if patt == None:
        patt = Patient(PatientID = int(dt_dict['PatientID']))

    if dt_dict['DataType'] in [ 4 ] :
        WriteTempItem(dt_dict, session, patt)
        
    session_add(session, patt)
    session.commit()
    
session.commit()

[OrderedDict([('OEORDRowId', '4263350||1'), ('ItemName', '招工体检(无乙肝检查)'), ('ItemCat', None), ('InstrDR', None), ('DoseQty', None), ('PHFreqDR', None), ('Remark', '1'), ('Status', '已执行'), ('PrescNo', None), ('Priority', None), ('ToldType', '临时医嘱'), ('YJDate', '2017-07-21'), ('Date', None), ('Loc', '体检保健科'), ('RecLoc', '体检保健科'), ('Price', '152.4'), ('Unit', None), ('RecLocID', None), ('StDate', '2017-07-21'), ('EtDate', None), ('RecLocDesc', None), ('NeedExecTime', None), ('CommonName', None), ('YPFlag', '0')])]
[OrderedDict([('OEORDRowId', '4264353||1'), ('ItemName', '司机体检'), ('ItemCat', None), ('InstrDR', None), ('DoseQty', None), ('PHFreqDR', None), ('Remark', '1'), ('Status', '已执行'), ('PrescNo', None), ('Priority', None), ('ToldType', '临时医嘱'), ('YJDate', '2017-07-21'), ('Date', None), ('Loc', '体检保健科'), ('RecLoc', '体检保健科'), ('Price', '25'), ('Unit', None), ('RecLocID', None), ('StDate', '2017-07-21'), ('EtDate', None), ('RecLocDesc', None), ('NeedExecTime', None), ('CommonName', None),

## Type 5 EHR

## Overall Processing

In [5]:
dataType = df[df['DataType'] == 5 ]
s = set()
for ind, dt in dataType.iterrows():
    dt_dict = dt.to_dict()
    dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
    Data = dt_dict['Data']
    if 'EPRdata' in Data:
        EPR = Data['EPRdata']
        if EPR == None:
            #print(Data)
            continue
        l = ' '.join( i for i in list(EPR.keys()))
        s.add(l)
        
        if 'hourRec' in EPR:
            a = EPR['hourRec']
            b = EPR
            dt_dict_test = dt_dict
            break
a

OrderedDict([('A1', '入院记录'),
             ('A2', '24小时内入出院记录'),
             ('ChiefComplaint', '反复便后肛门肿物脱出肛外伴便血1余年，加重10天。'),
             ('Ininfo',
              '青年女性，慢性病程。反复便后肛门肿物脱出肛外伴便血1余年，加重10天。肿物逐渐增大致大便时脱出，便后不能自行回纳，少量带血，血色鲜红。肛检（截石位）：肛缘可见环状皮赘突起，齿线粘膜隆起并与皮赘相连，肛门括约肌收缩功能正常，指检直肠下段未触及肿块，指套无染血。辅助检查：肛门镜检查直肠下段粘膜无糜烂、出血及溃疡。未发现肿物。'),
             ('Indiag', '环状混合痔'),
             ('Treatment',
              '入院后完善术前常规检查；血常规，凝血四项未见明显异常。拟明日送手术室在腰麻下行混合痔切除术。患者因个人原因，要求出院，已告知可能出现情况于患者，表示理解仍要求出院，予带药办理出院。'),
             ('Outdiag', '环状混合痔'),
             ('Outoeord',
              '避免大便干结，注意清淡饮食，保持大便通畅，一个月内避免重体力劳动，尽量使用坐厕。不适随诊，如有便血立即返院复查。')])

In [6]:
dt_dict_test 

{'Data': OrderedDict([('EPRdata',
               OrderedDict([('RANinfo',
                             OrderedDict([('A1', '入院记录'),
                                          ('A2', '入院记录'),
                                          ('Basicinfo',
                                           OrderedDict([('ChiefComplaint',
                                                         '主诉:反复便后肛门肿物脱出肛外伴便血1余年，加重10天。'),
                                                        ('Pasthistory',
                                                         '既往史:2016年7月曾在外院行剖宫产手术；既往体健，否认“高血压，冠心病，糖尿病”等重大内脏疾病史；否认“肝炎，结核”等传染病史；否认外伤史；否认食物药物过敏史；常规预防接种；否认输血史。'),
                                                        ('Personalhistory',
                                                         '个人史:原籍出生长大，现深圳生活。无居住地地方病情况，无疫水、有害气体、物品、放射线接触史，无吸烟饮酒及其他不良嗜好，否认冶游史，否认吸毒史，无传染病接触史，无食鱼生史。'),
                                                        ('Obstericalhistory',
                                                         '婚

## dt_dict

In [7]:
EPR = dt_dict_test['Data']['EPRdata']
EPR

OrderedDict([('RANinfo',
              OrderedDict([('A1', '入院记录'),
                           ('A2', '入院记录'),
                           ('Basicinfo',
                            OrderedDict([('ChiefComplaint',
                                          '主诉:反复便后肛门肿物脱出肛外伴便血1余年，加重10天。'),
                                         ('Pasthistory',
                                          '既往史:2016年7月曾在外院行剖宫产手术；既往体健，否认“高血压，冠心病，糖尿病”等重大内脏疾病史；否认“肝炎，结核”等传染病史；否认外伤史；否认食物药物过敏史；常规预防接种；否认输血史。'),
                                         ('Personalhistory',
                                          '个人史:原籍出生长大，现深圳生活。无居住地地方病情况，无疫水、有害气体、物品、放射线接触史，无吸烟饮酒及其他不良嗜好，否认冶游史，否认吸毒史，无传染病接触史，无食鱼生史。'),
                                         ('Obstericalhistory',
                                          '婚育史:已婚，育有一子。家属均体健。'),
                                         ('Menstrualhistory',
                                          '月经史:初潮年龄13岁，周期28－30天，经期5－7天，末次月经日期2017年12月20日，颜色暗红，经量中等，无痛经史，白带正常。'),
                  

In [8]:
EPR.keys()

odict_keys(['RANinfo', 'FirstCorinfo', 'DailyRecdata', 'GetDisRecordinfo', 'hourRec'])

##### RANinfo

In [9]:
EPR['RANinfo']

OrderedDict([('A1', '入院记录'),
             ('A2', '入院记录'),
             ('Basicinfo',
              OrderedDict([('ChiefComplaint', '主诉:反复便后肛门肿物脱出肛外伴便血1余年，加重10天。'),
                           ('Pasthistory',
                            '既往史:2016年7月曾在外院行剖宫产手术；既往体健，否认“高血压，冠心病，糖尿病”等重大内脏疾病史；否认“肝炎，结核”等传染病史；否认外伤史；否认食物药物过敏史；常规预防接种；否认输血史。'),
                           ('Personalhistory',
                            '个人史:原籍出生长大，现深圳生活。无居住地地方病情况，无疫水、有害气体、物品、放射线接触史，无吸烟饮酒及其他不良嗜好，否认冶游史，否认吸毒史，无传染病接触史，无食鱼生史。'),
                           ('Obstericalhistory', '婚育史:已婚，育有一子。家属均体健。'),
                           ('Menstrualhistory',
                            '月经史:初潮年龄13岁，周期28－30天，经期5－7天，末次月经日期2017年12月20日，颜色暗红，经量中等，无痛经史，白带正常。'),
                           ('Familyhistory',
                            '家族史:家族中无类似疾病患者。否认有家族性遗传、免疫性和精神性疾病患者。')])),
             ('PhysicalExamination',
              '发育正常，营养中等，正常体型，自动体位，表情安静，面色红润，意识清醒，姿势步态正常，语调与语态正常， 精神状态一般，检查合作，回答切题，无恶病质。皮肤粘膜色泽正常，未见皮疹。无出血点、紫癜、瘀斑、血肿；无粘膜溃疡、皮下结节、

In [10]:
if 'RANinfo' in EPR:
    RAN = EPR['RANinfo']

    if 'Basicinfo' in RAN:
        Basicinfo = RAN.pop('Basicinfo')
        if type(Basicinfo) == str:
            Basicinfo = '<xml>'  + Basicinfo+ '</xml>'  
            result = xmltodict.parse(Basicinfo)
            Basicinfo = result['xml']
        RAN_info = RANinfo(**Basicinfo, **RAN)
        
    else:
        RAN_info = RANinfo(**RAN)
    
RAN_info.__dict__

{'A1': '入院记录',
 'A2': '入院记录',
 'Accessoryexamination': '肛镜检直肠下段粘膜无充血、水肿、糜烂、溃疡、出血点、肿物。',
 'ChiefComplaint': '主诉:反复便后肛门肿物脱出肛外伴便血1余年，加重10天。',
 'Familyhistory': '家族史:家族中无类似疾病患者。否认有家族性遗传、免疫性和精神性疾病患者。',
 'Menstrualhistory': '月经史:初潮年龄13岁，周期28－30天，经期5－7天，末次月经日期2017年12月20日，颜色暗红，经量中等，无痛经史，白带正常。',
 'Obstericalhistory': '婚育史:已婚，育有一子。家属均体健。',
 'Pasthistory': '既往史:2016年7月曾在外院行剖宫产手术；既往体健，否认“高血压，冠心病，糖尿病”等重大内脏疾病史；否认“肝炎，结核”等传染病史；否认外伤史；否认食物药物过敏史；常规预防接种；否认输血史。',
 'Personalhistory': '个人史:原籍出生长大，现深圳生活。无居住地地方病情况，无疫水、有害气体、物品、放射线接触史，无吸烟饮酒及其他不良嗜好，否认冶游史，否认吸毒史，无传染病接触史，无食鱼生史。',
 'PhysicalExamination': '发育正常，营养中等，正常体型，自动体位，表情安静，面色红润，意识清醒，姿势步态正常，语调与语态正常， 精神状态一般，检查合作，回答切题，无恶病质。皮肤粘膜色泽正常，未见皮疹。无出血点、紫癜、瘀斑、血肿；无粘膜溃疡、皮下结节、蜘蛛痣、尿霜、色素沉着、色素脱失。全身浅表淋巴结无肿大，头颅大小及形状正常，颜面部无浮肿，结膜无充血，巩膜无黄染，双侧瞳孔等圆等大，对光反射良好，双侧耳廓正常，外耳道无异常分泌物，乳突无压痛，听力正常，无鼻翼扇动，鼻腔通气良好，副鼻窦无压痛，口唇无发绀，口腔无特殊气味，口腔粘膜无溃疡，牙龈无出血，无龋齿，咽部无充血，扁桃体无肿大，颈部双侧对称，颈静脉无怒张，无肝颈静脉回流征，气管正中，甲状腺无肿大，胸廓对称无畸形，肋间隙正常，双肺呼吸运动对称，语颤正常，无胸膜摩擦感,胸壁无压痛，胸骨无叩痛，双肺叩诊清音，双肺呼吸音清晰，未闻及干湿性罗音和胸膜摩擦音。心前区无隆起及凹陷，心尖搏动位于第五肋间左锁骨中线内1cm

##### FirstCorinfo

In [12]:
FCI = EPR['FirstCorinfo']
FCI

OrderedDict([('A1', '病程记录'),
             ('A2', '首次病程记录'),
             ('Basicinfo',
              '患者何静，女，27岁，因“反复便后肛门肿物脱出肛外伴便血1余年，加重10天。”于2017年1月12日 11:25:00收入本区。'),
             ('Characteristics',
              '1、患者为青年女性，慢性病程。\n  2、患者1年前无诱因开始出现大便后肿物脱出肛门外，伴出血，偶伴疼痛，曾在外院门诊诊断“混合痔”，予以治疗（具体不详）。之后肿物逐渐增大致大便时脱出，便后不能自行回纳，少量带血，血色鲜红，未予重视及治疗；10天前患者诉上述症状加重，伴疼痛不适，今来我科就诊，门诊拟“环状混合痔”收入院。患者发病以来无畏寒、发热、腹胀、腹痛等不适，精神一般，食欲正常，睡眠正常，大便通畅，小便正常，体重无变化。\n  3、查体：T：36.1℃，P：79次/分，呼吸：20次/分，血压：115/61mmHg。\n   发育正常，营养中等，正常体型，自动体位，表情安静，面色红润，意识清醒，姿势步态正常，语调与语态正常， 精神状态一般，检查合作，回答切题，无恶病质。皮肤粘膜色泽正常，未见皮疹。无出血点、紫癜、瘀斑、血肿；无粘膜溃疡、皮下结节、蜘蛛痣、尿霜、色素沉着、色素脱失。全身浅表淋巴结无肿大，头颅大小及形状正常，颜面部无浮肿，结膜无充血，巩膜无黄染，双侧瞳孔等圆等大，对光反射良好，双侧耳廓正常，外耳道无异常分泌物，乳突无压痛，听力正常，无鼻翼扇动，鼻腔通气良好，副鼻窦无压痛，口唇无发绀，口腔无特殊气味，口腔粘膜无溃疡，牙龈无出血，无龋齿，咽部无充血，扁桃体无肿大，颈部双侧对称，颈静脉无怒张，无肝颈静脉回流征，气管正中，甲状腺无肿大，胸廓对称无畸形，肋间隙正常，双肺呼吸运动对称，语颤正常，无胸膜摩擦感,胸壁无压痛，胸骨无叩痛，双肺叩诊清音，双肺呼吸音清晰，未闻及干湿性罗音和胸膜摩擦音。心前区无隆起及凹陷，心尖搏动位于第五肋间左锁骨中线内1cm，搏动范围正常，心前区未触及震颤和心包摩擦感，心相对浊音界正常，心率：79次／分,心律齐，各瓣膜区未闻及心脏杂音。腹部平坦，中下腹可见手术疤痕，腹式呼吸存在，无腹壁静脉曲张，未见肠型及蠕动波；

In [14]:
data = FCI['Basicinfo']

name = data.split('，')[0].replace('患者', '')
FCI['Basicinfo'] = data.replace(name, 'XX')
FCI['Basicinfo']

'患者XX，女，27岁，因“反复便后肛门肿物脱出肛外伴便血1余年，加重10天。”于2017年1月12日 11:25:00收入本区。'

In [27]:
if 'FirstCorinfo' in EPR:
    FCI = EPR['FirstCorinfo']
    FCI_info = FirstCorinfo(**FCI)
    

    
FCI_info.__dict__

{'A1': '病程记录',
 'A2': '首次病程记录',
 'Basicinfo': '患者何静，女，27岁，因“反复便后肛门肿物脱出肛外伴便血1余年，加重10天。”于2017年1月12日 11:25:00收入本区。',
 'Characteristics': '1、患者为青年女性，慢性病程。\n  2、患者1年前无诱因开始出现大便后肿物脱出肛门外，伴出血，偶伴疼痛，曾在外院门诊诊断“混合痔”，予以治疗（具体不详）。之后肿物逐渐增大致大便时脱出，便后不能自行回纳，少量带血，血色鲜红，未予重视及治疗；10天前患者诉上述症状加重，伴疼痛不适，今来我科就诊，门诊拟“环状混合痔”收入院。患者发病以来无畏寒、发热、腹胀、腹痛等不适，精神一般，食欲正常，睡眠正常，大便通畅，小便正常，体重无变化。\n  3、查体：T：36.1℃，P：79次/分，呼吸：20次/分，血压：115/61mmHg。\n   发育正常，营养中等，正常体型，自动体位，表情安静，面色红润，意识清醒，姿势步态正常，语调与语态正常， 精神状态一般，检查合作，回答切题，无恶病质。皮肤粘膜色泽正常，未见皮疹。无出血点、紫癜、瘀斑、血肿；无粘膜溃疡、皮下结节、蜘蛛痣、尿霜、色素沉着、色素脱失。全身浅表淋巴结无肿大，头颅大小及形状正常，颜面部无浮肿，结膜无充血，巩膜无黄染，双侧瞳孔等圆等大，对光反射良好，双侧耳廓正常，外耳道无异常分泌物，乳突无压痛，听力正常，无鼻翼扇动，鼻腔通气良好，副鼻窦无压痛，口唇无发绀，口腔无特殊气味，口腔粘膜无溃疡，牙龈无出血，无龋齿，咽部无充血，扁桃体无肿大，颈部双侧对称，颈静脉无怒张，无肝颈静脉回流征，气管正中，甲状腺无肿大，胸廓对称无畸形，肋间隙正常，双肺呼吸运动对称，语颤正常，无胸膜摩擦感,胸壁无压痛，胸骨无叩痛，双肺叩诊清音，双肺呼吸音清晰，未闻及干湿性罗音和胸膜摩擦音。心前区无隆起及凹陷，心尖搏动位于第五肋间左锁骨中线内1cm，搏动范围正常，心前区未触及震颤和心包摩擦感，心相对浊音界正常，心率：79次／分,心律齐，各瓣膜区未闻及心脏杂音。腹部平坦，中下腹可见手术疤痕，腹式呼吸存在，无腹壁静脉曲张，未见肠型及蠕动波；无压痛、反跳痛，未触及包块。肝右肋下未触及，脾左肋下未触及，肝区无叩击痛，未触及胆囊，Murphy征阴性，脾浊音区正常。双肋脊角及腰部无隆起，压痛（－），双侧输尿管点

##### DailyRecdata

In [23]:
EPR['DailyRecdata']

OrderedDict([('A1', '病程记录'),
             ('A2', '日常病程记录'),
             ('once',
              [OrderedDict([('happentime', '2017-01-12 12:26:02'),
                            ('title', '主治医师查房记录'),
                            ('Text',
                             '青年女性，慢性病程。反复便后肛门肿物脱出肛外伴便血1余年，加重10天。肿物逐渐增大致大便时脱出，便后不能自行回纳，少量带血，血色鲜红。肛检（截石位）：肛缘可见环状皮赘突起，齿线粘膜隆起并与皮赘相连，肛门括约肌收缩功能正常，指检直肠下段未触及肿块，指套无染血。辅助检查：肛门镜检查直肠下段粘膜无糜烂、出血及溃疡。未发现肿物。冯能宙主治医师查房指示：根据症状、体征诊断：混合痔；完善与手术相关各项检查如：血常规，凝血四项、胸片、心电图等。明日手术治疗。')]),
               OrderedDict([('happentime', '2017-01-12 16:26:30'),
                            ('title', '主任医师查房记录'),
                            ('Text',
                             '青年女性，慢性病程。反复便后肛门肿物脱出肛外伴便血1余年，加重10天。肿物逐渐增大致大便时脱出，便后不能自行回纳，少量带血，血色鲜红。肛检（截石位）：肛缘可见环状皮赘突起，齿线粘膜隆起并与皮赘相连，肛门括约肌收缩功能正常，指检直肠下段未触及肿块，指套无染血。辅助检查：肛门镜检查直肠下段粘膜无糜烂、出血及溃疡。未发现肿物。吴先哲主任医师查房指示：根据症状、体征诊断：环状混合痔。本病应与肛管癌鉴别，肛管癌患者肛门持续性疼痛，便后加重，早期少量便血并逐渐加重，大便习惯改变，次数增多，排粪不尽感，直肠指检可扪及肿块。故可鉴别；建议尽快完善与手术相关各项检查，如无特殊手术禁忌症，拟明日在腰麻下行混合痔切除术。患者因个人原因，要求今日出院，

In [24]:
if 'DailyRecdata' in EPR:
    DRe = EPR['DailyRecdata']
    if 'once' in DRe:
        DailyRecs = DRe.pop('once')
        if type(DailyRecs) != list:
            DailyRecs = [DailyRecs]
    else:
        DailyRecs =[]
        
    for DailyRec in DailyRecs:
        DRe_data = DailyRecdata(**DRe, **DailyRec)
        

In [26]:
DRe_data.__dict__

{'A1': '病程记录',
 'A2': '日常病程记录',
 'Text': '青年女性，慢性病程。反复便后肛门肿物脱出肛外伴便血1余年，加重10天。肿物逐渐增大致大便时脱出，便后不能自行回纳，少量带血，血色鲜红。肛检（截石位）：肛缘可见环状皮赘突起，齿线粘膜隆起并与皮赘相连，肛门括约肌收缩功能正常，指检直肠下段未触及肿块，指套无染血。辅助检查：肛门镜检查直肠下段粘膜无糜烂、出血及溃疡。未发现肿物。吴先哲主任医师查房指示：根据症状、体征诊断：环状混合痔。本病应与肛管癌鉴别，肛管癌患者肛门持续性疼痛，便后加重，早期少量便血并逐渐加重，大便习惯改变，次数增多，排粪不尽感，直肠指检可扪及肿块。故可鉴别；建议尽快完善与手术相关各项检查，如无特殊手术禁忌症，拟明日在腰麻下行混合痔切除术。患者因个人原因，要求今日出院，已告知可能出现情况于患者，表示理解仍要求出院，劝解无效，予带药办理出院。',
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x19af22ba8>,
 'happentime': '2017-01-12 16:26:30',
 'title': '主任医师查房记录'}

##### GetDisRecordinfo

In [28]:
EPR['GetDisRecordinfo']

OrderedDict([('A1', '出院记录'),
             ('A2', '出院记录'),
             ('Indiag', '环状混合痔'),
             ('Treatment',
              '入院后完善术前常规检查；血常规，凝血四项、胸片、心电图未见明显异常。送手术室在腰麻下行环状混合痔切除术/内痔环切术/肛周脓肿I期根治术/肛裂切除术/肛瘘切开旷置术/尖锐湿疣电灼术。术程顺利，术中出血不多，麻醉满意，术后安返病房。术后给予抗炎、止血、换药等对症支持治疗。'),
             ('Outdiag', '环状混合痔')])

In [34]:
if 'GetDisRecordinfo' in EPR:
    GDR = EPR['GetDisRecordinfo']
    GDR_info = GetDisRecordinfo(**GDR)

    
GDR_info.__dict__

{'A1': '出院记录',
 'A2': '出院记录',
 'Indiag': '环状混合痔',
 'Outdiag': '环状混合痔',
 'Treatment': '入院后完善术前常规检查；血常规，凝血四项、胸片、心电图未见明显异常。送手术室在腰麻下行环状混合痔切除术/内痔环切术/肛周脓肿I期根治术/肛裂切除术/肛瘘切开旷置术/尖锐湿疣电灼术。术程顺利，术中出血不多，麻醉满意，术后安返病房。术后给予抗炎、止血、换药等对症支持治疗。',
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x19a2cb400>}

###### hourRec

In [35]:
EPR['hourRec']

OrderedDict([('A1', '入院记录'),
             ('A2', '24小时内入出院记录'),
             ('ChiefComplaint', '反复便后肛门肿物脱出肛外伴便血1余年，加重10天。'),
             ('Ininfo',
              '青年女性，慢性病程。反复便后肛门肿物脱出肛外伴便血1余年，加重10天。肿物逐渐增大致大便时脱出，便后不能自行回纳，少量带血，血色鲜红。肛检（截石位）：肛缘可见环状皮赘突起，齿线粘膜隆起并与皮赘相连，肛门括约肌收缩功能正常，指检直肠下段未触及肿块，指套无染血。辅助检查：肛门镜检查直肠下段粘膜无糜烂、出血及溃疡。未发现肿物。'),
             ('Indiag', '环状混合痔'),
             ('Treatment',
              '入院后完善术前常规检查；血常规，凝血四项未见明显异常。拟明日送手术室在腰麻下行混合痔切除术。患者因个人原因，要求出院，已告知可能出现情况于患者，表示理解仍要求出院，予带药办理出院。'),
             ('Outdiag', '环状混合痔'),
             ('Outoeord',
              '避免大便干结，注意清淡饮食，保持大便通畅，一个月内避免重体力劳动，尽量使用坐厕。不适随诊，如有便血立即返院复查。')])

In [36]:
if 'hourRec' in EPR:
    HRe = EPR['hourRec']
    HRe_data = hourRec(**HRe)
    
HRe_data.__dict__

{'A1': '入院记录',
 'A2': '24小时内入出院记录',
 'ChiefComplaint': '反复便后肛门肿物脱出肛外伴便血1余年，加重10天。',
 'Indiag': '环状混合痔',
 'Ininfo': '青年女性，慢性病程。反复便后肛门肿物脱出肛外伴便血1余年，加重10天。肿物逐渐增大致大便时脱出，便后不能自行回纳，少量带血，血色鲜红。肛检（截石位）：肛缘可见环状皮赘突起，齿线粘膜隆起并与皮赘相连，肛门括约肌收缩功能正常，指检直肠下段未触及肿块，指套无染血。辅助检查：肛门镜检查直肠下段粘膜无糜烂、出血及溃疡。未发现肿物。',
 'Outdiag': '环状混合痔',
 'Outoeord': '避免大便干结，注意清淡饮食，保持大便通畅，一个月内避免重体力劳动，尽量使用坐厕。不适随诊，如有便血立即返院复查。',
 'Treatment': '入院后完善术前常规检查；血常规，凝血四项未见明显异常。拟明日送手术室在腰麻下行混合痔切除术。患者因个人原因，要求出院，已告知可能出现情况于患者，表示理解仍要求出院，予带药办理出院。',
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x19af16ef0>}

###### OperRecData

In [39]:
dataType = df[df['DataType'] == 5 ]
s = set()
for ind, dt in dataType.iterrows():
    dt_dict = dt.to_dict()
    dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
    Data = dt_dict['Data']
    if 'EPRdata' in Data:
        EPR = Data['EPRdata']
        if EPR == None:
            #print(Data)
            continue
        l = ' '.join( i for i in list(EPR.keys()))
        s.add(l)
        
        if 'OperRecdata' in EPR:
            a = EPR['OperRecdata']
            b = EPR
            dt_dict_test = dt_dict
            break
a

OrderedDict([('A1', '手术相关'),
             ('A2', '手术记录'),
             ('once',
              OrderedDict([('happentime', '2017-07-24 19:36:49'),
                           ('title', '手术过程'),
                           ('OperTime', '2017-07-24 16:35'),
                           ('OperName', '经腹子宫肌瘤剔除术+左侧输卵管系膜囊肿切除'),
                           ('Text',
                            '患者患者取截石位，气管插管全麻成功后，常规消毒铺巾。取下腹正中纵行切口长约12cm，首先剔除原手术瘢痕，并等长逐层切开皮下脂肪等直达腹膜，术者洗手探查：双侧输卵管卵巢未见异常，左侧输卵管系膜囊肿一枚直径0.3cm，切除送病理检查。子宫增大形态失常，浆膜下肌壁间可见多发肌性结节，右侧宫角部完全凸向浆膜下的分叶状肌瘤结节直径约7cm；子宫前壁肌壁间肌瘤结节直径约5cm，宫后壁体部及下段各一个肌瘤结节，直径分别为3cm及2cm。常规行肌瘤剔除术。结合术前彩超提示有一直径约2cm之肌瘤结节完全位于粘膜下，仔细触摸宫腔内一2cm的结节。子宫前壁肌瘤结节剔除后瘤床尚有肌层，切开一小口，钳夹宫腔内粘膜下肌瘤，扭转摘除，创面无出血。3/0可吸收线缝合前壁内膜关闭宫腔。所有创面双层缝合关闭。检查创面无出血，再次触摸子宫未及异常隆起结节。清点纱布器械无误，用生理盐水冲洗盆腔，仔细检查各创面无出血后，留置强生可吸收防粘连膜于子宫前后壁创面预防粘连。逐层关腹术终，无菌敷料覆盖切口，手术结束。\n\n术程顺利，麻醉满意，生命体征平稳，术中出血约80ml，补液1000ml，尿量100ml，色清亮，术后安返病房。\n\n共切除肌瘤结节5枚及输卵管系膜囊肿1个，标本家属过目后送病理检查。')]))])

In [40]:
EPR = dt_dict_test['Data']['EPRdata']
EPR

OrderedDict([('RANinfo',
              OrderedDict([('A1', '入院记录'),
                           ('A2', '入院记录'),
                           ('Basicinfo',
                            OrderedDict([('ChiefComplaint', '主诉:发现盆腔包块3+年。'),
                                         ('Pasthistory',
                                          '既往史:既往健康，1996年行剖宫产术，2015年行左侧乳腺纤维瘤切除术，否认“结核、病毒性肝炎、血吸虫病”等传染病史，否认“慢性支气管炎、高血压、心脏病、冠心病、肾病、糖尿病”等慢性病史，无重大外伤输血史，无食物及药物过敏史。预防接种史不详。'),
                                         ('Personalhistory',
                                          '个人史:原籍出生长大，无烟、酒、药等嗜好。否认工业毒物、粉尘、放射性物质接触史，否认疫区居住史及疫水接触史，否认冶游史及性病史。'),
                                         ('Obstericalhistory',
                                          '婚育史:结婚年龄23岁，配偶年龄：47岁，健康状况：良好，治疗情况：无。'),
                                         ('Menstrualhistory',
                                          '月经史:初潮年龄14岁，周期23天，经期7天，经量偏多，无痛经史,LMP:2017年7月17日。'),
                                         ('Familyhistory',
          

In [41]:
if 'OperRecdata' in EPR:
    ORd = EPR['OperRecdata']
    if 'once' in ORd:
        OperRecs = ORd.pop('once')
        if type(OperRecs) != list:
            OperRecs = [OperRecs]
    else:
        OperRecs =[]
        
    for OperRec in OperRecs:
        OperRec = OperRecdata(**ORd, **OperRec)
        
OperRec.__dict__

{'A1': '手术相关',
 'A2': '手术记录',
 'OperName': '经腹子宫肌瘤剔除术+左侧输卵管系膜囊肿切除',
 'OperTime': '2017-07-24 16:35',
 'Text': '患者患者取截石位，气管插管全麻成功后，常规消毒铺巾。取下腹正中纵行切口长约12cm，首先剔除原手术瘢痕，并等长逐层切开皮下脂肪等直达腹膜，术者洗手探查：双侧输卵管卵巢未见异常，左侧输卵管系膜囊肿一枚直径0.3cm，切除送病理检查。子宫增大形态失常，浆膜下肌壁间可见多发肌性结节，右侧宫角部完全凸向浆膜下的分叶状肌瘤结节直径约7cm；子宫前壁肌壁间肌瘤结节直径约5cm，宫后壁体部及下段各一个肌瘤结节，直径分别为3cm及2cm。常规行肌瘤剔除术。结合术前彩超提示有一直径约2cm之肌瘤结节完全位于粘膜下，仔细触摸宫腔内一2cm的结节。子宫前壁肌瘤结节剔除后瘤床尚有肌层，切开一小口，钳夹宫腔内粘膜下肌瘤，扭转摘除，创面无出血。3/0可吸收线缝合前壁内膜关闭宫腔。所有创面双层缝合关闭。检查创面无出血，再次触摸子宫未及异常隆起结节。清点纱布器械无误，用生理盐水冲洗盆腔，仔细检查各创面无出血后，留置强生可吸收防粘连膜于子宫前后壁创面预防粘连。逐层关腹术终，无菌敷料覆盖切口，手术结束。\n\n术程顺利，麻醉满意，生命体征平稳，术中出血约80ml，补液1000ml，尿量100ml，色清亮，术后安返病房。\n\n共切除肌瘤结节5枚及输卵管系膜囊肿1个，标本家属过目后送病理检查。',
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x198c66278>,
 'happentime': '2017-07-24 19:36:49',
 'title': '手术过程'}

##### OpEMRData

In [48]:
dt = df[df['DataType'] == 5].iloc[11]
dt_dict = dt.to_dict()
dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
dt_dict

{'Data': OrderedDict([('EPRdata',
               OrderedDict([('OpEMRData',
                             OrderedDict([('A1', '门诊病历'),
                                          ('A2', '门诊病历'),
                                          ('Basicinfo',
                                           '主诉：头冲地摔伤颈椎痛1月，查体：颈椎活动痛，'),
                                          ('Accessoryexamination',
                                           '左侧肩关节正侧位诊断意见：左肩关节骨质未见异常,请随诊。\nCT：颈椎未见异常'),
                                          ('Tentativediagnosis', '1.(颈外伤)'),
                                          ('TreatMent',
                                           'CT颈椎椎体平扫(3个椎体)＋三维成像 组  \n休息\n双氯芬酸钠双释放肠溶胶囊(戴芬) 75mg 口服 qd骨康胶囊 1.2g 口服 tid')]))]))]),
 'DataType': 5,
 'Id': 6944,
 'JobNumber': 4265613,
 'JobOrgan_Id': 1,
 'PatientID': 35153,
 'RecipeID': 731,
 'RegiNumber': 175791}

In [49]:
EPR = dt_dict['Data']['EPRdata']
EPR

OrderedDict([('OpEMRData',
              OrderedDict([('A1', '门诊病历'),
                           ('A2', '门诊病历'),
                           ('Basicinfo', '主诉：头冲地摔伤颈椎痛1月，查体：颈椎活动痛，'),
                           ('Accessoryexamination',
                            '左侧肩关节正侧位诊断意见：左肩关节骨质未见异常,请随诊。\nCT：颈椎未见异常'),
                           ('Tentativediagnosis', '1.(颈外伤)'),
                           ('TreatMent',
                            'CT颈椎椎体平扫(3个椎体)＋三维成像 组  \n休息\n双氯芬酸钠双释放肠溶胶囊(戴芬) 75mg 口服 qd骨康胶囊 1.2g 口服 tid')]))])

In [50]:
if 'OpEMRData' in EPR:
    OEM = EPR['OpEMRData']
    OEM_data = OpEMRData(**OEM)
    
OEM_data.__dict__

{'A1': '门诊病历',
 'A2': '门诊病历',
 'Accessoryexamination': '左侧肩关节正侧位诊断意见：左肩关节骨质未见异常,请随诊。\nCT：颈椎未见异常',
 'Basicinfo': '主诉：头冲地摔伤颈椎痛1月，查体：颈椎活动痛，',
 'Tentativediagnosis': '1.(颈外伤)',
 'TreatMent': 'CT颈椎椎体平扫(3个椎体)＋三维成像 组  \n休息\n双氯芬酸钠双释放肠溶胶囊(戴芬) 75mg 口服 qd骨康胶囊 1.2g 口服 tid',
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x19a819240>}

##### DeathRecordData

In [53]:
dataType = df[df['DataType'] == 5 ]
s = set()
for ind, dt in dataType.iterrows():
    dt_dict = dt.to_dict()
    dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
    Data = dt_dict['Data']
    if 'EPRdata' in Data:
        EPR = Data['EPRdata']
        if EPR == None:
            #print(Data)
            continue
        l = ' '.join( i for i in list(EPR.keys()))
        s.add(l)
        
        if 'DeathRecordinfo' in EPR:
            a = EPR['DeathRecordinfo']
            b = EPR
            dt_dict_test = dt_dict
            break
a

OrderedDict([('A1', '出院记录'),
             ('A2', '死亡记录'),
             ('Ininfo',
              '患者中年男性，急性病程。因“咳嗽、呛咳、咳痰伴呼吸困难3天”入院。既往史：2013年发现肺癌，2014年在广州市肿瘤医院行化疗及“右下肺癌根治术”治疗，术后病理诊断为鳞癌。2014-2015年间分别在江西省肿瘤医院及广东省人民医院肿瘤科住院行放化疗。患者因肺癌转移食管致管腔狭窄致吞咽困难，于15天前在香港医院行食管支架置入术治疗，因气管狭窄，10天前在广州呼吸疾病研究所行气道支架置入术治疗。有“高血压”病史，服用“美托洛尔缓释片、络活喜”降压治疗。患者儿时因病致双眼失明。入院查体：体温36.6℃，心率93次/分，血压139/89mmHg，呼吸22次/分，血氧饱和度73%。呼吸促，口唇轻度发绀，咽部明显充血，双侧扁桃体无肿大，胸廓对称无畸形，肋间隙正常，右胸背部可见一长约20cm手术瘢痕，双肺呼吸运动对称，右肺语颤较左肺减低，无胸膜摩擦感，胸壁无压痛，胸骨无叩痛，右肺叩诊呈浊音，左肺叩诊清音，右肺呼吸音低，左肺呼吸音稍粗，双下肺闻及湿罗音，右肺明显，双肺未闻及干性罗音及胸膜摩擦音。心相对浊音界正常，心率93次/分，心律齐，各瓣膜区未闻及心脏杂音。辅助检查：2015-05-09我院门诊血常规示白细胞计数9.15X10^9/L、中性粒细胞比值89.4%、血红蛋白浓度82g/L，CRP160mg/L，胸部CT示肺癌术后，左支气管、食道支架置入术后改变，双肺感染，左肺多发结节转移性病灶，右侧胸膜肥厚、粘连伴钙化并包裹性积液，心包积液；纵隔内多发肿大淋巴结，主动脉粥样硬化，附见：右肾囊肿。'),
             ('Indiag',
              '1、肺部感染,2、呼吸衰竭,3、右下肺鳞癌术后并纵隔淋巴结、胸膜、肺内、食管、骨转移,4、左主支气管支架置入术后,5、食道支架置入术后,6、高血压病，7、轻度贫血。'),
             ('Treatment',
              '患者住院期间完善相关检查。血常规血小板计数:546x10~9/L,血红蛋白浓度:85g/L,红细胞压积:0.263L/L,平均RBC体积:78.9fl,平均RBC血红

In [54]:
EPR = dt_dict_test['Data']['EPRdata']
EPR

OrderedDict([('RANinfo',
              OrderedDict([('A1', '入院记录'),
                           ('A2', '入院记录'),
                           ('Basicinfo',
                            OrderedDict([('ChiefComplaint',
                                          '主诉:咳嗽、呛咳、咳痰伴呼吸困难3天'),
                                         ('Pasthistory',
                                          '既往史:2013年发现肺癌，2014年在广州市肿瘤医院行化疗及“右下肺癌根治术”治疗，术后病理诊断为鳞癌。2014-2015年间分别在江西省肿瘤医院及广东省人民医院肿瘤科住院行放化疗。患者因肺癌转移食管致管腔狭窄致吞咽困难，于15天前在香港医院行食管支架置入术治疗，因气管狭窄，在广州呼吸疾病研究所行气道支架置入术治疗。有“高血压”病史，服用“美托洛尔缓释片、络活喜”降压治疗。否认“冠心病、肾病、糖尿病”等慢性病史，否认“结核、病毒性肝炎、肝吸虫病、血吸虫病”等传染病史，否认重大外伤史及输血史。否认食物及药物过敏史。预防接种史不详。'),
                                         ('Personalhistory',
                                          '个人史:出生于江西省，居住于深圳市，吸烟30年，2包/日，已戒烟。偶有饮酒，无毒物放射性物质接触史，无疫区疫区、疫水接触史，无食鱼生史，否认冶游史，否认吸毒史。'),
                                         ('Obstericalhistory',
                                          '婚育史:已婚，配偶健康状况：良好，夫妻感情好；生育状况：育有3个子女。'),
                  

In [55]:
EPR.keys()

odict_keys(['RANinfo', 'FirstCorinfo', 'DailyRecdata', 'DeathRecordinfo'])

In [56]:
if 'DeathRecordinfo' in EPR:
    DRe = EPR['DeathRecordinfo']
    DRe_info = DeathRecordinfo(**DRe)   
    
DRe_info.__dict__

{'A1': '出院记录',
 'A2': '死亡记录',
 'Deathdiag': '1、肺部感染,2、Ⅰ型呼吸衰竭,3、肺癌晚期（鳞癌）,4、左主支气管支架置入术后,5、食道支架置入术后,6、高血压病，7、轻度贫血，8、低蛋白血症。',
 'Indiag': '1、肺部感染,2、呼吸衰竭,3、右下肺鳞癌术后并纵隔淋巴结、胸膜、肺内、食管、骨转移,4、左主支气管支架置入术后,5、食道支架置入术后,6、高血压病，7、轻度贫血。',
 'Ininfo': '患者中年男性，急性病程。因“咳嗽、呛咳、咳痰伴呼吸困难3天”入院。既往史：2013年发现肺癌，2014年在广州市肿瘤医院行化疗及“右下肺癌根治术”治疗，术后病理诊断为鳞癌。2014-2015年间分别在江西省肿瘤医院及广东省人民医院肿瘤科住院行放化疗。患者因肺癌转移食管致管腔狭窄致吞咽困难，于15天前在香港医院行食管支架置入术治疗，因气管狭窄，10天前在广州呼吸疾病研究所行气道支架置入术治疗。有“高血压”病史，服用“美托洛尔缓释片、络活喜”降压治疗。患者儿时因病致双眼失明。入院查体：体温36.6℃，心率93次/分，血压139/89mmHg，呼吸22次/分，血氧饱和度73%。呼吸促，口唇轻度发绀，咽部明显充血，双侧扁桃体无肿大，胸廓对称无畸形，肋间隙正常，右胸背部可见一长约20cm手术瘢痕，双肺呼吸运动对称，右肺语颤较左肺减低，无胸膜摩擦感，胸壁无压痛，胸骨无叩痛，右肺叩诊呈浊音，左肺叩诊清音，右肺呼吸音低，左肺呼吸音稍粗，双下肺闻及湿罗音，右肺明显，双肺未闻及干性罗音及胸膜摩擦音。心相对浊音界正常，心率93次/分，心律齐，各瓣膜区未闻及心脏杂音。辅助检查：2015-05-09我院门诊血常规示白细胞计数9.15X10^9/L、中性粒细胞比值89.4%、血红蛋白浓度82g/L，CRP160mg/L，胸部CT示肺癌术后，左支气管、食道支架置入术后改变，双肺感染，左肺多发结节转移性病灶，右侧胸膜肥厚、粘连伴钙化并包裹性积液，心包积液；纵隔内多发肿大淋巴结，主动脉粥样硬化，附见：右肾囊肿。',
 'Reasonofdeath': '呼吸循环衰竭。',
 'Treatment': '患者住院期间完善相关检查。血常规血小板计数:546x10~9/L,血红蛋白浓度:85g/L,红细胞压积:0.263L/L,平均RBC体积:78.9

### Overall function

In [8]:
from cctmodels import EPR as EPRecord
from cctmodels import OpEMRData, RANinfo, FirstCorinfo, DailyRecdata, GetDisRecordinfo, DeathRecordinfo, OperRecdata, hourRec


def WriteEPR(dt_dict, session, patt):

    eprRec = EPRecord()
    
    EPR = dt_dict['Data']['EPRdata']
    if EPR == None:
        print(dt_dict)
        return 
    
    if 'OpEMRData' in EPR:
        OEM = EPR['OpEMRData']
        OEM_data = OpEMRData(**OEM)
        eprRec.OpEMRDatas.append(OEM_data)
        session_add(session, OEM_data)

    if 'RANinfo' in EPR:
        RAN = EPR['RANinfo']

        if 'Basicinfo' in RAN:
            Basicinfo = RAN.pop('Basicinfo')
            if type(Basicinfo) == str:
                Basicinfo = '<xml>'  + Basicinfo+ '</xml>'  
                result = xmltodict.parse(Basicinfo)
                Basicinfo = result['xml']
            RAN_info = RANinfo(**Basicinfo, **RAN)

        else:
            RAN_info = RANinfo(**RAN)
            
        eprRec.RANinfos.append(RAN_info)
        session_add(session, RAN_info)
    
    if 'FirstCorinfo' in EPR:
        FCI = EPR['FirstCorinfo']
        FCI_info = FirstCorinfo(**FCI)
        eprRec.FirstCorinfos.append(FCI_info)
        session_add(session, FCI_info)
            
    if 'DailyRecdata' in EPR:
        DRe = EPR['DailyRecdata']
        if 'once' in DRe:
            DailyRecs = DRe.pop('once')
            if type(DailyRecs) != list:
                DailyRecs = [DailyRecs]
        else:
            DailyRecs =[]
        
        for DailyRec in DailyRecs:
            DRe_data = DailyRecdata(**DRe, **DailyRec)
            eprRec.DailyRecdatas.append(DRe_data)
            session_add(session, DRe_data)
        
    if 'OperRecdata' in EPR:
        ORd = EPR['OperRecdata']
        if 'once' in ORd:
            OperRecs = ORd.pop('once')
            if type(OperRecs) != list:
                OperRecs = [OperRecs]
        else:
            OperRecs =[]

        for OperRec in OperRecs:
            OperRec = OperRecdata(**ORd, **OperRec)
            eprRec.OperRecdatas.append(OperRec)
            session_add(session, OperRec)

    if 'GetDisRecordinfo' in EPR:
        GDR = EPR['GetDisRecordinfo']
        GDR_info = GetDisRecordinfo(**GDR)
        eprRec.GetDisRecordinfos.append(GDR_info)
        session_add(session, GDR_info)

    if 'hourRec' in EPR:
        HRe = EPR['hourRec']
        HRe_data = hourRec(**HRe)
        eprRec.hourRecs.append(HRe_data)
        session_add(session, HRe_data)
        
    if 'DeathRecordinfo' in EPR:
        DRe = EPR['DeathRecordinfo']
        DRe_info = DeathRecordinfo(**DRe)  
        eprRec.DeathRecordinfos.append(DRe_info)
        session_add(session, DRe_info)
    
    patt.EPRs.append(eprRec)
    session_add(session, eprRec)
    # print(n)

In [9]:
dt = df[df['DataType'] == 5].iloc[121]
dt_dict = dt.to_dict()
dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
dt_dict

{'Data': OrderedDict([('EPRdata',
               OrderedDict([('RANinfo',
                             OrderedDict([('A1', '入院记录'),
                                          ('A2', '入院记录'),
                                          ('Basicinfo',
                                           OrderedDict([('ChiefComplaint',
                                                         '主诉:停经40+6周，入院待产。'),
                                                        ('Pasthistory',
                                                         '既往史:无过敏史、输血史、手术史；无心脏病、高血压、肝病、肾病、糖尿病、血液病、结核病、骨关节病等疾病史。'),
                                                        ('Personalhistory',
                                                         '个人史:原籍成长，否认疫水、疫区接触史，否认有冶游史。否认有毒化学物质及放射线接触史。无不良生活嗜好。不嗜烟，不嗜酒。预防接种史不详。否认有食物药物过敏史。'),
                                                        ('Obstericalhistory',
                                                         '婚育史:结婚年龄24岁，配偶年龄：30岁，健康状况：良好。'),
                             

In [10]:
patt = session.query(Patient).filter_by(PatientID = int(dt_dict['PatientID'])).first()
if patt == None:
    patt = Patient(PatientID = int(dt_dict['PatientID']))
    
    
WriteEPR(dt_dict, session, patt)


session_add(session, patt)
session.commit()

In [13]:
dataType = df[df['DataType'] == 9 ]
for ind, dt in dataType.iterrows():
    dt_dict = dt.to_dict()
    dt_dict['Data'] = xmltodict.parse(dt_dict['Data'] )
    
    patt = session.query(Patient).filter_by(PatientID = int(dt_dict['PatientID'])).first()
    if patt == None:
        patt = Patient(PatientID = int(dt_dict['PatientID']))

    if dt_dict['DataType'] in [ 5, 9 ] :
        WriteEPR(dt_dict, session, patt)
        
    session_add(session, patt)
    session.commit()
    
session.commit()

{'Id': 19440, 'RecipeID': 4851, 'PatientID': 36594, 'JobNumber': 2958805, 'RegiNumber': 249030, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19444, 'RecipeID': 4852, 'PatientID': 30796, 'JobNumber': 2959199, 'RegiNumber': 783423, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19448, 'RecipeID': 4853, 'PatientID': 27552, 'JobNumber': 2959366, 'RegiNumber': 501928, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19776, 'RecipeID': 4943, 'PatientID': 24746, 'JobNumber': 2932669, 'RegiNumber': 37960, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19780, 'RecipeID': 4945, 'PatientID': 34718, 'JobNumber': 2933113, 'RegiNumber': 145253, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19784, 'RecipeID': 4946, 'PatientID': 28338, 'JobNumber': 2926224, 'RegiNumber': 689286, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDi

{'Id': 24692, 'RecipeID': 6305, 'PatientID': 26045, 'JobNumber': 2456999, 'RegiNumber': 35746, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 24828, 'RecipeID': 6343, 'PatientID': 24099, 'JobNumber': 2441597, 'RegiNumber': 677995, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 24832, 'RecipeID': 6344, 'PatientID': 20489, 'JobNumber': 2441618, 'RegiNumber': 257625, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 25128, 'RecipeID': 6425, 'PatientID': 27806, 'JobNumber': 2419184, 'RegiNumber': 655996, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 25156, 'RecipeID': 6432, 'PatientID': 38498, 'JobNumber': 2415544, 'RegiNumber': 47821, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 25168, 'RecipeID': 6435, 'PatientID': 27394, 'JobNumber': 2415814, 'RegiNumber': 209645, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDic

{'Id': 19548, 'RecipeID': 4881, 'PatientID': 22928, 'JobNumber': 2947252, 'RegiNumber': 85134, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19468, 'RecipeID': 4858, 'PatientID': 33010, 'JobNumber': 2954541, 'RegiNumber': 742391, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19472, 'RecipeID': 4859, 'PatientID': 36594, 'JobNumber': 2954886, 'RegiNumber': 249030, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19476, 'RecipeID': 4860, 'PatientID': 26726, 'JobNumber': 2955203, 'RegiNumber': 132207, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19480, 'RecipeID': 4861, 'PatientID': 32958, 'JobNumber': 2955788, 'RegiNumber': 329075, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19556, 'RecipeID': 4883, 'PatientID': 30177, 'JobNumber': 2948706, 'RegiNumber': 76401, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDic

{'Id': 19800, 'RecipeID': 4950, 'PatientID': 33067, 'JobNumber': 2926864, 'RegiNumber': 97867, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19804, 'RecipeID': 4951, 'PatientID': 27742, 'JobNumber': 2927281, 'RegiNumber': 713537, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19816, 'RecipeID': 4954, 'PatientID': 37813, 'JobNumber': 2928713, 'RegiNumber': 37081, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19808, 'RecipeID': 4952, 'PatientID': 27882, 'JobNumber': 2927357, 'RegiNumber': 106440, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19812, 'RecipeID': 4953, 'PatientID': 36887, 'JobNumber': 2927817, 'RegiNumber': 166204, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 19820, 'RecipeID': 4955, 'PatientID': 24746, 'JobNumber': 2928819, 'RegiNumber': 37960, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict

{'Id': 20176, 'RecipeID': 5051, 'PatientID': 36864, 'JobNumber': 2893677, 'RegiNumber': 737373, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 20180, 'RecipeID': 5052, 'PatientID': 27508, 'JobNumber': 2893767, 'RegiNumber': 71057, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 20184, 'RecipeID': 5053, 'PatientID': 35322, 'JobNumber': 2893986, 'RegiNumber': 263573, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 20292, 'RecipeID': 5084, 'PatientID': 32737, 'JobNumber': 2875859, 'RegiNumber': 196042, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 20188, 'RecipeID': 5054, 'PatientID': 21969, 'JobNumber': 2895423, 'RegiNumber': 520482, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 20192, 'RecipeID': 5056, 'PatientID': 25853, 'JobNumber': 2889170, 'RegiNumber': 432711, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDi

...

{'Id': 58747, 'RecipeID': 14330, 'PatientID': 29608, 'JobNumber': 56928, 'RegiNumber': 2119, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 58751, 'RecipeID': 14331, 'PatientID': 24767, 'JobNumber': 56977, 'RegiNumber': 9063, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 58807, 'RecipeID': 14345, 'PatientID': 24338, 'JobNumber': 53113, 'RegiNumber': 29459, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 58811, 'RecipeID': 14346, 'PatientID': 24762, 'JobNumber': 51943, 'RegiNumber': 6822, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 58775, 'RecipeID': 14337, 'PatientID': 35443, 'JobNumber': 55399, 'RegiNumber': 30373, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 58779, 'RecipeID': 14338, 'PatientID': 31937, 'JobNumber': 55490, 'RegiNumber': 30413, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata'

{'Id': 59115, 'RecipeID': 14422, 'PatientID': 34380, 'JobNumber': 18558, 'RegiNumber': 12728, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 59123, 'RecipeID': 14424, 'PatientID': 32555, 'JobNumber': 17754, 'RegiNumber': 12300, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 59127, 'RecipeID': 14425, 'PatientID': 32623, 'JobNumber': 17512, 'RegiNumber': 12148, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 59131, 'RecipeID': 14426, 'PatientID': 28688, 'JobNumber': 17516, 'RegiNumber': 12150, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 59135, 'RecipeID': 14427, 'PatientID': 32623, 'JobNumber': 17575, 'RegiNumber': 12148, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 59139, 'RecipeID': 14428, 'PatientID': 22476, 'JobNumber': 17595, 'RegiNumber': 12197, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRda

{'Id': 59379, 'RecipeID': 14488, 'PatientID': 38529, 'JobNumber': 4363111, 'RegiNumber': 343830, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 59383, 'RecipeID': 14489, 'PatientID': 35472, 'JobNumber': 4364065, 'RegiNumber': 57970, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 59515, 'RecipeID': 14522, 'PatientID': 21986, 'JobNumber': 4371436, 'RegiNumber': 57329, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 59519, 'RecipeID': 14523, 'PatientID': 23394, 'JobNumber': 4371812, 'RegiNumber': 956239, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 59391, 'RecipeID': 14491, 'PatientID': 39427, 'JobNumber': 4364920, 'RegiNumber': 151923, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 59411, 'RecipeID': 14496, 'PatientID': 27201, 'JobNumber': 4365423, 'RegiNumber': 59836, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': Order

{'Id': 60331, 'RecipeID': 14725, 'PatientID': 26075, 'JobNumber': 4434439, 'RegiNumber': 399031, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 60343, 'RecipeID': 14728, 'PatientID': 28771, 'JobNumber': 4436504, 'RegiNumber': 846674, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 60347, 'RecipeID': 14729, 'PatientID': 28771, 'JobNumber': 4436581, 'RegiNumber': 846674, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 61931, 'RecipeID': 15125, 'PatientID': 36054, 'JobNumber': 4553126, 'RegiNumber': 277008, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 61935, 'RecipeID': 15126, 'PatientID': 32279, 'JobNumber': 4553348, 'RegiNumber': 1099646, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 60363, 'RecipeID': 14733, 'PatientID': 38778, 'JobNumber': 4437990, 'RegiNumber': 132942, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': O

{'Id': 61259, 'RecipeID': 14957, 'PatientID': 21490, 'JobNumber': 4510821, 'RegiNumber': 1110954, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 61291, 'RecipeID': 14965, 'PatientID': 28207, 'JobNumber': 4511698, 'RegiNumber': 1002405, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 61335, 'RecipeID': 14976, 'PatientID': 38005, 'JobNumber': 4515287, 'RegiNumber': 42259, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 61299, 'RecipeID': 14967, 'PatientID': 35742, 'JobNumber': 4511939, 'RegiNumber': 284156, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 61303, 'RecipeID': 14968, 'PatientID': 29674, 'JobNumber': 4511943, 'RegiNumber': 514586, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': OrderedDict([('EPRdata', None)])}
{'Id': 61307, 'RecipeID': 14969, 'PatientID': 23394, 'JobNumber': 4512063, 'RegiNumber': 956239, 'JobOrgan_Id': 1, 'DataType': 9, 'Data': O

### Data Reports

In [30]:
df = pd.read_json('data/xml_table.json', orient = 'split')
df

,Id,RecipeID,PatientID,JobNumber,RegiNumber,JobOrgan_Id,DataType,Data
0,6702,281,31571,3168507,130256,1,5,<EPRdata><RANinfo><A1>入院记录</A1><A2>入院记录</A2><B...
1,6716,475,31571,1653681,130256,1,5,<EPRdata><RANinfo><A1>入院记录</A1><A2>入院记录</A2><B...
2,11584,2088,26309,3789068,945512,1,7,<Request><HisID/><Name/><Sex>女</Sex><Age>49岁</...
3,6707,395,31571,2115881,130256,1,3,<Request><HisID/><Name/><Sex>男</Sex><Age>82岁</...
4,6718,629,31571,764829,130256,1,3,<Request><HisID/><Name/><Sex>男</Sex><Age>82岁</...
5,6728,672,24531,4310335,1064503,1,2,<Request><Name/><Sex>男</Sex><Age>40岁</Age><InA...
6,6733,673,29880,4310690,1064543,1,4,<Request><TempItemInfo><TempItemInfo><OEORDRow...
7,6759,682,33079,4311843,1064634,1,4,<Request><TempItemInfo><TempItemInfo><OEORDRow...
8,6836,703,25562,4271524,51918,1,2,<Request><Name/><Sex>男</Sex><Age>11岁</Age><InA...
9,6774,686,39491,4312335,71295,1,4,<Request><TempItemInfo><TempItemInfo><OEORDRow...


In [43]:
dataType = df[df['DataType'] == 8]
dataType

,Id,RecipeID,PatientID,JobNumber,RegiNumber,JobOrgan_Id,DataType,Data
16,7170,796,29131,4263281,86393,1,8,<Request><TempItemInfo><TempItemInfo><OEORDRow...
22,7574,909,38542,4224594,236125,1,8,<Request><TempItemInfo><TempItemInfo><OEORDRow...
31,7672,935,32724,4209111,185371,1,8,<Request><TempItemInfo><TempItemInfo><OEORDRow...
34,7880,996,21455,4166109,736262,1,8,<Request><TempItemInfo><TempItemInfo><OEORDRow...
41,8396,1142,24746,4107744,37960,1,8,<Request><TempItemInfo><TempItemInfo><OEORDRow...
44,8480,1165,29801,4095718,1009786,1,8,<Request><TempItemInfo><TempItemInfo><OEORDRow...
46,8550,1185,30194,4095134,912546,1,8,<Request><TempItemInfo><TempItemInfo><OEORDRow...
55,9111,1353,21466,4028220,998430,1,8,<Request><TempItemInfo><TempItemInfo><OEORDRow...
61,9450,1448,36603,3997823,991730,1,8,<Request><TempItemInfo><TempItemInfo><OEORDRow...
63,9676,1513,20431,3976312,897411,1,8,<Request><TempItemInfo><TempItemInfo><OEORDRow...


In [36]:
dataType[dataType['Data'] == '<EPRdata/>']

,Id,RecipeID,PatientID,JobNumber,RegiNumber,JobOrgan_Id,DataType,Data
39647,48565,794,28646,4262961,75753,1,5,<EPRdata/>
39652,48570,845,36845,4247106,205093,1,5,<EPRdata/>
39653,48571,846,23898,4247174,45641,1,5,<EPRdata/>
39787,48625,1179,20945,4098962,72465,1,5,<EPRdata/>
39838,48626,1181,24305,4094350,127557,1,5,<EPRdata/>
39840,48629,1198,23791,4087736,522296,1,5,<EPRdata/>
39842,48631,1211,29149,4084904,332085,1,5,<EPRdata/>
39852,48646,1328,33064,4034790,203386,1,5,<EPRdata/>
39868,48648,1341,37239,4038136,1000660,1,5,<EPRdata/>
39876,48664,1455,21169,3999335,520937,1,5,<EPRdata/>


In [25]:
dataType = df[df['DataType'] == 5 ]
dataType[dataType['Data'] != '<EPRdata/>']

,Id,RecipeID,PatientID,JobNumber,RegiNumber,JobOrgan_Id,DataType,Data
0,6702,281,31571,3168507,130256,1,5,<EPRdata><RANinfo><A1>入院记录</A1><A2>入院记录</A2><B...
1,6716,475,31571,1653681,130256,1,5,<EPRdata><RANinfo><A1>入院记录</A1><A2>入院记录</A2><B...
30,6842,704,24839,4271682,706798,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
4990,7011,750,28322,4267054,322908,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5039,6900,719,37955,4264265,1052883,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5043,6904,720,20858,4264273,762140,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5067,6951,733,28963,4265735,339134,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5108,7018,752,37999,4267247,965967,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5130,6853,707,20858,4272290,762140,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5138,6877,713,21523,4272637,1055101,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...


In [26]:
dataType = df[df['DataType'] == 5 ]
dataType

,Id,RecipeID,PatientID,JobNumber,RegiNumber,JobOrgan_Id,DataType,Data
0,6702,281,31571,3168507,130256,1,5,<EPRdata><RANinfo><A1>入院记录</A1><A2>入院记录</A2><B...
1,6716,475,31571,1653681,130256,1,5,<EPRdata><RANinfo><A1>入院记录</A1><A2>入院记录</A2><B...
30,6842,704,24839,4271682,706798,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
4990,7011,750,28322,4267054,322908,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5039,6900,719,37955,4264265,1052883,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5043,6904,720,20858,4264273,762140,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5067,6951,733,28963,4265735,339134,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5108,7018,752,37999,4267247,965967,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5130,6853,707,20858,4272290,762140,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
5138,6877,713,21523,4272637,1055101,1,5,<EPRdata><OpEMRData><A1>门诊病历</A1><A2>门诊病历</A2>...
